In [2]:
import pandas as pd 
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import resample


In [3]:
df = pd.read_csv("../data/clean/df_processed.csv")
df.drop(['Unnamed: 0','race'],inplace=True,axis=1)

In [4]:
#taking a sample of data which has equal destrubution of tatget column values 
sample_df = df.groupby('readmitted').apply(lambda x: x.sample(n=50,random_state=2,replace=True))

/var/folders/q1/bq_b0f_d53ldgrm4rh3vh1_40000gn/T/ipykernel_20263/654470899.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_df = df.groupby('readmitted').apply(lambda x: x.sample(n=50,random_state=2,replace=True))


In [5]:
#creating a new df with the remaining data
remaining_df = pd.concat([df,sample_df ]).drop_duplicates(keep=False)

In [6]:
print(f'{df.shape}')
print(f'{sample_df.shape}')
print(f'{remaining_df.shape}')

(1043, 24)
(150, 24)
(906, 24)


In [7]:
# For training data
X_train = sample_df.drop(columns=['readmitted'])
y_train = sample_df['readmitted']

# For testing data
X_test = remaining_df.drop(columns=['readmitted'])
y_test = remaining_df['readmitted']

In [8]:
X_train_cat = X_train.select_dtypes('object')
X_test_cat  = X_test.select_dtypes('object')
#
X_train_num = X_train.select_dtypes('number')
X_test_num  = X_test.select_dtypes('number')

In [9]:
categorical_column_values = [ df[col].unique() for col in df.select_dtypes('object').columns ]

In [10]:
encoder = OneHotEncoder(drop='first',handle_unknown='ignore', sparse_output=False)

In [11]:
encoder.fit(X_train_cat)

OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)

In [12]:
X_train_cat_encoded_np = encoder.transform(X_train_cat)
X_test_cat_encoded_np  = encoder.transform(X_test_cat)

/Users/og/Desktop/IronHackDA/7-Final_Project/SugarSync/sugarsync/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [1, 3, 5, 6, 7, 8, 9] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [13]:
#categorical_column_values[0] = np.append(categorical_column_values[0], 'asian')
categorical_column_values[0]

array(['female', 'male'], dtype=object)

In [14]:
X_test_cat_encoded_np[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.])

In [15]:
X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_train.index)
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_test.index)

In [16]:
X_train = pd.concat([X_train_num, X_train_cat_encoded_df], axis=1)
X_test = pd.concat([X_test_num, X_test_cat_encoded_df], axis=1)

In [17]:
normalizer = MinMaxScaler()
normalizer.fit(X_train)

MinMaxScaler()

In [18]:
X_train_norm_np = normalizer.transform(X_train)
X_test_norm_np = normalizer.transform(X_test)

In [19]:
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [20]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [21]:
dt_predictions = dt_classifier.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_predictions)
print("Decision Tree Accuracy:", dt_accuracy)

Decision Tree Accuracy: 0.3543046357615894


In [22]:
rf_predictions = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

Random Forest Accuracy: 0.3708609271523179
